<a href="https://colab.research.google.com/github/Xuehui-Jiang/Uni-Mannheim-Master-Thesis/blob/main/Experimental_Evaluation/Experiment_Movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cloning the pyRDF2Vec repo
!git clone https://github.com/IBCNServices/pyRDF2Vec.git

Cloning into 'pyRDF2Vec'...
remote: Enumerating objects: 7412, done.
remote: Counting objects: 100% (380/380), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 7412 (delta 234), reused 259 (delta 133), pack-reused 7032
Receiving objects: 100% (7412/7412), 5.32 MiB | 11.23 MiB/s, done.
Resolving deltas: 100% (5038/5038), done.


In [ ]:
!pip install rdflib
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip uninstall community
!pip install python-louvain
!pip install pyRDF2Vec --upgrade

Found existing installation: community 1.0.0b1
Uninstalling community-1.0.0b1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/community-1.0.0b1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/community/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/community/community_louvain.py
    /usr/local/lib/python3.10/dist-packages/community/community_status.py
Proceed (Y/n)? Y
  Successfully uninstalled community-1.0.0b1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import random
import warnings
warnings.filterwarnings('ignore')
import functools
import itertools
from typing import List, Sequence, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rdflib
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC

import sys
sys.path.append('pyRDF2Vec')
sys.path.append('pyRDF2Vec/pyrdf2vec')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.walkers import RandomWalker

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


Saving completeDataset.tsv to completeDataset.tsv
User uploaded file "completeDataset.tsv" with length 182653 bytes


In [ ]:
# load training and testing entities
complete_df = pd.read_csv('completeDataset.tsv', sep='\t')

entities = complete_df['DBpedia_URI'].tolist()
labels = complete_df['Label'].tolist()
reg_targets = complete_df['rating'].tolist()

In [ ]:
# Define the list of files
files_to_merge = [
    "instance_types_en.ttl",
    "infobox_properties_en.ttl",
    "article_categories_en.ttl",
    "disambiguations_en.ttl"
]

# Merge files
with open("direct_merged_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open("/content/drive/My Drive/dbpedia_files/" + file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


In [ ]:
import shutil

destination_path = "/content/drive/My Drive/dbpedia_files"

shutil.copy("indirect_merged_dbpedia.ttl", destination_path)

print(f"File saved to {destination_path}")

File saved to /content/drive/My Drive/dbpedia_files


In [ ]:
# Define the list of files
files_to_merge = [
    "mappingbased_objects_en.ttl",
    "geonames_links_en.ttl"
]

# Merge files
with open("indirect_merged_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open("/content/drive/My Drive/dbpedia_files/" + file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


In [ ]:
# Filter out entities and relationships directly related to Movies Dataset
with open("/content/drive/My Drive/dbpedia_files/direct_merged_dbpedia.ttl", "r") as file, open("direct_movies_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2:
            # Check if the subject or object of the triple is in our city URL list
            if triple[0].strip('<>') in entities or triple[2].strip('<>') in entities:
                out_file.write(line)

In [ ]:
# Filter out entities and relationships directly or indirectly related to Cities Dataset
entities_related_to_cities = set(entities)
new_entities = set(entities)
iteration_count = 0

while new_entities and iteration_count < 2:
    temp_entities = set()
    with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file:
        for line in file:
            triple = line.strip().split()
            if len(triple) > 2 and (triple[0].strip('<>') in new_entities or triple[2].strip('<>') in new_entities):
                    temp_entities.add(triple[0].strip('<>'))
                    temp_entities.add(triple[2].strip('<>'))

    new_entities = temp_entities - entities_related_to_cities
    entities_related_to_cities.update(new_entities)
    iteration_count += 1  # 每次循环完成后增加计数器

# Finally, we can write all triples that are directly or indirectly related to entities in the `entities_related_to_cities` collection
with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file, open("indirect_movies_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2 and (triple[0].strip('<>') in entities_related_to_cities or triple[2].strip('<>') in entities_related_to_cities):
            out_file.write(line)


In [ ]:
# Define the list of files
files_to_merge = [
    "direct_movies_filtered_dbpedia.ttl",
    "indirect_movies_filtered_dbpedia.ttl"
]

# Merge files
with open("movies_filtered_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open(file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


In [ ]:
input_file = "movies_filtered_dbpedia.ttl"
output_file = "cleaned_movies_filtered_dbpedia.ttl"

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Determine whether it is a comment or a blank line
        if line.startswith("#") or line.strip() == "":
            continue

        # Split triples
        triple = line.strip().split()

        # Make sure this is a complete triple
        if len(triple) > 2:
            # 检查对象是否以“http://dbpedia.org/resource/”开头
            if triple[2].startswith("<http://dbpedia.org/resource/"):
                outfile.write(line)

print(f"Finished filtering {input_file} and saved the result to {output_file}")


Finished filtering movies_filtered_dbpedia.ttl and saved the result to cleaned_movies_filtered_dbpedia.ttl


In [ ]:
!head -n 10 cleaned_movies_filtered_dbpedia.ttl

<http://dbpedia.org/resource/Blade_Runner> <http://dbpedia.org/property/productionCompanies> <http://dbpedia.org/resource/The_Ladd_Company> .
<http://dbpedia.org/resource/Blade_Runner> <http://dbpedia.org/property/productionCompanies> <http://dbpedia.org/resource/Shaw_Brothers_Studio> .
<http://dbpedia.org/resource/Batman_&_Robin_(film)> <http://dbpedia.org/property/studio> <http://dbpedia.org/resource/PolyGram_Filmed_Entertainment> .
<http://dbpedia.org/resource/Dr._Strangelove> <http://dbpedia.org/property/studio> <http://dbpedia.org/resource/Hawk_Films> .
<http://dbpedia.org/resource/Das_Boot> <http://dbpedia.org/property/productionCompanies> <http://dbpedia.org/resource/Bavaria_Film> .
<http://dbpedia.org/resource/Das_Boot> <http://dbpedia.org/property/productionCompanies> <http://dbpedia.org/resource/Westdeutscher_Rundfunk> .
<http://dbpedia.org/resource/Das_Boot> <http://dbpedia.org/property/productionCompanies> <http://dbpedia.org/resource/SWR_Fernsehen> .
<http://dbpedia.org/re

In [ ]:
import shutil

destination_path = "/content/drive/My Drive/dbpedia_files"

shutil.copy("cleaned_movies_filtered_dbpedia.ttl", destination_path)

print(f"File saved to {destination_path}")

File saved to /content/drive/My Drive/dbpedia_files


In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("/content/drive/My Drive/dbpedia_files/cleaned_movies_filtered_dbpedia.ttl", fmt='turtle')

kgentities = kg._entities

In [ ]:
for e in entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

http://dbpedia.org/resource/Moolaad%C3%A9
http://dbpedia.org/resource/Bad_Education_(film)_(film)
http://dbpedia.org/resource/The_Raid:_Redemption
http://dbpedia.org/resource/Fados
http://dbpedia.org/resource/What_a_Man_(film)
http://dbpedia.org/resource/Thunderbirds_(film)
http://dbpedia.org/resource/Jumper_(film)
http://dbpedia.org/resource/ATM_(film)
http://dbpedia.org/resource/Knife_Fight
http://dbpedia.org/resource/Zoom_(film)
http://dbpedia.org/resource/Short_Cuts
http://dbpedia.org:8890/resource/Lone_Star_(1996_film)
http://dbpedia.org/resource/La_grande_strada_azzurra
http://dbpedia.org/resource/Big_Animal
http://dbpedia.org/resource/The_Lookout
http://dbpedia.org/resource/The_Proposition
http://dbpedia.org/resource/Good_Hair_(film)
http://dbpedia.org/resource/Intimate_Strangers
http://dbpedia.org/resource/Alice_and_Martin_(1998_film)
http://dbpedia.org/resource/Open_Range
http://dbpedia.org/resource/All_the_Right_Moves_(film)_(film)
http://dbpedia.org/resource/Alexander_(film)

In [ ]:
# Entities not in KG
absent_entities = [
    "http://dbpedia.org/resource/Moolaad%C3%A9",
    "http://dbpedia.org/resource/Bad_Education_(film)_(film)",
    "http://dbpedia.org/resource/The_Raid:_Redemption",
    "http://dbpedia.org/resource/Fados",
    "http://dbpedia.org/resource/What_a_Man_(film)",
    "http://dbpedia.org/resource/Thunderbirds_(film)",
    "http://dbpedia.org/resource/Jumper_(film)",
    "http://dbpedia.org/resource/ATM_(film)",
    "http://dbpedia.org/resource/Knife_Fight",
    "http://dbpedia.org/resource/Zoom_(film)",
    "http://dbpedia.org/resource/Short_Cuts",
    "http://dbpedia.org:8890/resource/Lone_Star_(1996_film)",
    "http://dbpedia.org/resource/La_grande_strada_azzurra",
    "http://dbpedia.org/resource/Big_Animal",
    "http://dbpedia.org/resource/The_Lookout",
    "http://dbpedia.org/resource/The_Proposition",
    "http://dbpedia.org/resource/Good_Hair_(film)",
    "http://dbpedia.org/resource/Intimate_Strangers",
    "http://dbpedia.org/resource/Alice_and_Martin_(1998_film)",
    "http://dbpedia.org/resource/Open_Range",
    "http://dbpedia.org/resource/All_the_Right_Moves_(film)_(film)",
    "http://dbpedia.org/resource/Alexander_(film)",
    "http://dbpedia.org/resource/Gigantic_(film)",
    "http://dbpedia.org/resource/Dream_House_(film)",
    "http://dbpedia.org/resource/Dreamcatcher_(film)",
    "http://dbpedia.org/resource/Man_of_the_House_(2005_crime_comedy_film)",
    "http://dbpedia.org/resource/Play_the_Game_(film)",
    "http://dbpedia.org/resource/Sphere_(film)",
    "http://dbpedia.org/resource/Excess_Baggage",
    "http://dbpedia.org/resource/Motherhood_(film)",
    "http://dbpedia.org/resource/As_Luck_Would_Have_It_(2012_film)_(2012_film)",
    "http://dbpedia.org/resource/Das_Wilde_Leben",
    "http://dbpedia.org/resource/Someone_Like_You_(film)",
    "http://dbpedia.org/resource/John_Q",
    "http://dbpedia.org/resource/The_Divide_(film)",
    "http://dbpedia.org/resource/Jack_and_Jill_(film)",
    "http://dbpedia.org/resource/Molly_(film)",
    "http://dbpedia.org/resource/The_Wash_(film)",
    "http://dbpedia.org/resource/Followers",
    "http://dbpedia.org/resource/Carpool_(film)",
    "http://dbpedia.org/resource/Tomcats_(film)"
]

# Filter out absent entities
filtered_entities = [e for e in entities if e not in absent_entities]
filtered_labels = [label for entity, label in zip(entities, labels) if entity not in absent_entities]
filtered_reg_targets = [reg_targets for entity, reg_targets in zip(entities, reg_targets) if entity not in absent_entities]

Prepare for Knowledge Graph

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
import networkx as nx

In [ ]:
# Load the graph from the input file
graph = Graph()
graph.parse("/content/drive/My Drive/dbpedia_files/cleaned_movies_filtered_dbpedia.ttl", format="turtle")

<Graph identifier=Nbe957ee8d9784275afdf66e5acec75ea (<class 'rdflib.graph.Graph'>)>

In [ ]:
G = nx.DiGraph()  # Create a directed graph

for s, p, o in graph:
    G.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G.nodes())
num_edges = len(G.edges())
print("The DBpedia knowledge graph as of 2016-10 has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The DBpedia knowledge graph as of 2016-10 has 3309931 nodes and 9529784 edges.


Classification & Regression tasks based graph embeddings without SA

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error
import pandas as pd

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('pure_class_reg_results.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.772391
1           cbow         50                 C4.5  Accuracy   0.727902
2           cbow         50            KNN (k=3)  Accuracy   0.812622
3           cbow         50                  SVM  Accuracy   0.821829
4           cbow         50    Linear Regression      RMSE  19.297329


Classification & Regression tasks based graph embeddings extracted on subgraph base on SA with BFS



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving main.py to main.py
Saving sa_helper.py to sa_helper.py


In [ ]:
from main import *
from sa_helper import *

In [ ]:
# Count the neighbor node number for each nodes in the graph
neighbor_counts = {node: len(list(G.neighbors(node))) for node in G.nodes()}

excl=False, pop=False

In [ ]:
subgraph_F_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub.nodes())
num_edges = len(G_sub.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 24039 nodes and 58989 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_ff.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.557923
1           cbow         50                 C4.5  Accuracy   0.671763
2           cbow         50            KNN (k=3)  Accuracy   0.733511
3           cbow         50                  SVM  Accuracy   0.742690
4           cbow         50    Linear Regression      RMSE  22.826486


excl=True, pop=False

In [ ]:
subgraph_T_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=True, pop=False)

In [ ]:
G_sub_tf = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_F:
    G_sub_tf.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tf.nodes())
num_edges = len(G_sub_tf.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 24255 nodes and 59289 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_tf.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_tf.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_tf.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.652865
1           cbow         50                 C4.5  Accuracy   0.707998
2           cbow         50            KNN (k=3)  Accuracy   0.747794
3           cbow         50                  SVM  Accuracy   0.811617
4           cbow         50    Linear Regression      RMSE  22.401318


excl=False, pop=True

In [ ]:
subgraph_F_T = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts, alpha=0.186,
                                        beta=0.006, max_hops=3, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=True)

In [ ]:
G_sub_ft = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_T:
    G_sub_ft.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ft.nodes())
num_edges = len(G_sub_ft.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 21028 nodes and 55246 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_T.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ft.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ft.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_ft.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.629419
1           cbow         50                 C4.5  Accuracy   0.683987
2           cbow         50            KNN (k=3)  Accuracy   0.737622
3           cbow         50                  SVM  Accuracy   0.750361
4           cbow         50    Linear Regression      RMSE  22.866688


excl=True, pop=True

In [ ]:
subgraph_T_T = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts, alpha=0.186,
                                        beta=0.006, max_hops=3, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=True, pop=True)

In [ ]:
G_sub_tt = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_T:
    G_sub_tt.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tt.nodes())
num_edges = len(G_sub_tt.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 21028 nodes and 55246 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_T.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_tt.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_tt.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_ttl_class_reg_results_tt.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.631463
1           cbow         50                 C4.5  Accuracy   0.678925
2           cbow         50            KNN (k=3)  Accuracy   0.739139
3           cbow         50                  SVM  Accuracy   0.754456
4           cbow         50    Linear Regression      RMSE  22.826206


Classification & Regression tasks based graph embeddings extracted on subgraph base on SA with DFS

excl=False, pop=False

In [ ]:
subgraph_F_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub_ff = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub_ff.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ff.nodes())
num_edges = len(G_sub_ff.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 21027 nodes and 55246 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_ff.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.625311
1           cbow         50                 C4.5  Accuracy   0.671769
2           cbow         50            KNN (k=3)  Accuracy   0.747831
3           cbow         50                  SVM  Accuracy   0.761609
4           cbow         50    Linear Regression      RMSE  22.743039


excl=True, pop=False

In [ ]:
subgraph_T_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.1, beta=0.009, max_depth=2, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=True, pop=False)

In [ ]:
G_sub_tf = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_F:
    G_sub_tf.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tf.nodes())
num_edges = len(G_sub_tf.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 21027 nodes and 55246 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_tf.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_tf.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_tf.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.637561
1           cbow         50                 C4.5  Accuracy   0.688082
2           cbow         50            KNN (k=3)  Accuracy   0.743203
3           cbow         50                  SVM  Accuracy   0.766172
4           cbow         50    Linear Regression      RMSE  23.174798


excl=False, pop=True

In [ ]:
subgraph_F_T = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=True)

In [ ]:
G_sub_ft = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_T:
    G_sub_ft.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ft.nodes())
num_edges = len(G_sub_ft.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 21027 nodes and 55246 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_T.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ft.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ft.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_ft.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.653898
1           cbow         50                 C4.5  Accuracy   0.699314
2           cbow         50            KNN (k=3)  Accuracy   0.776913
3           cbow         50                  SVM  Accuracy   0.756489
4           cbow         50    Linear Regression      RMSE  22.402919


excl=True, pop=True

In [ ]:
subgraph_T_T = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=True, pop=True)

In [ ]:
G_sub_tt = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_T:
    G_sub_tt.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tt.nodes())
num_edges = len(G_sub_tt.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 21027 nodes and 55246 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_T.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_tt.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_tt.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_ttl_class_reg_results_tt.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.646743
1           cbow         50                 C4.5  Accuracy   0.709001
2           cbow         50            KNN (k=3)  Accuracy   0.781507
3           cbow         50                  SVM  Accuracy   0.757002
4           cbow         50    Linear Regression      RMSE  21.809703
